# ACTIVIDAD #

##  Construye un detector de movimiento en una región de interés de la imagen marcada manualmente. Guarda 2 ó 3 segundos de la secuencia detectada en un archivo de vídeo. ###

La sucesión de pasos para aplicar una Región de interés en una imagen (ROI) es: crear un ROI en la imagen, realizar la operación que desea en esta subregión de la imagen, restablecer el ROI.

En este notebook veremos como aplicar un ROI siguiendo un esquema diferente al utilizado por el profesor ya que se basa en en la aplicación de trackers de objetos. 

Object Tracking es un mecanismo muy extendido en OpenCV que consiste en determinar una zona donde existe un objeto y frame a frame se estudia su posición.

Existen varios mecanismos de tracking[1](https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/). De los cuales es ha seleccionado uno solo, en concreto se utilizará DCF (Discriminative Correlation Filter.

### DCF

Los DCF obtienen muy buen rendimiengo en seguimiento a corto plazo. El mapa de confiabilidad espacial ajusta el soporte del filtro a la parte del objeto adecuada para el seguimiento. Esto permite ampliar la región de búsqueda y mejora el seguimiento de objetos no rectangulares.

In [ ]:
# Usamos tracker Discriminative Correlation Filter
tracker = cv2.TrackerCSRT_create()

(...)

# Iniciamos el tracker sobre el objeto
success = tracker.init(frame, bbox)

## Diseño

El diseño se basa en la implementación del seguimiento de objetos[2](https://learnopencv.com/object-tracking-using-opencv-cpp-python/) que es más rápida que la detección de objetos.

Una vez establecido un algoritmo de seguimiento se ha implementado la lógica de la región de interés (ROI). Para adecuarlo al sistema que utiliza el algoritmo (ya que no está incluido en el paquete umucv) se ha utilizado la generación del ROI con OpenCV. 

Se utiliza el primer frame de video para obtener un punto de selección del objeto, a continuación se deberá presionar `Enter` para finalizar la selección.

Llegados a este punto comienza a reproducirse el vídeo tomado por la cámara que nos permite ver en tiempo real la posición del objeto. La caja se redibuja en el objeto a cada frame.


In [ ]:
# Si tenemos trakeado el objeto
    if success:
        # Creamos la caja
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)

Para concluir se ha implementado la funcionalidad de grabar, que permite, pulsando la tecla `G` grabar una fracción de tiempo hasta que se pulse de nuevo `G`. La tecla de `ESC` permite finalizar el programa (y la grabación).

In [ ]:
    if grabar:
        capture.write(frame)
    cv.imshow("Tracking", frame)
    k = cv.waitKey(1) & 0xff
    if k == ord('g'):
        grabar = not grabar
    elif k == 27:
        break

In [2]:
from IPython.display import HTML

HTML("""
    <video alt="test" idth="320" height="240"  controls>
        <source src="../Images/2_ACTIVIDAD/ACTIVIDAD1.mp4" type="video/mp4">
    </video>
""")

##  Opcional: muestra el objeto seleccionado anulando el fondo. ###

Para este ejercicio me he basado en las ideas aportadas en el notebook `chroma`, concretamente en la sección final donde se comenta los métodos para modelar y eliminar el fondo de una imagen[3](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_video/py_bg_subtraction/py_bg_subtraction.html).

La idea es utilizar un subtractor que consiste en un algoritmo de detección de fondo. Cuando una imagen se estabilza el contenido que no se mueve en varios frames se considera fondo. Esto genera unos píxeles activos y otros inactivos. La idea de diseño consiste en obtener esos pixeles y aplicarlos sobre los frames como una máscara.

In [ ]:
fgbg = cv.createBackgroundSubtractorMOG2()

MOG2 es un algoritmo de segmentación de fondo / primer plano basado en mezclas gaussianas. Una característica importante de este algoritmo es que selecciona el número apropiado de distribución gaussiana para cada píxel.

MOG2 tiene la opción de seleccionar si se detectará la sombra o no. Si `detectShadows = True` (que es así por defecto), detecta y marca las sombras, pero reduce la velocidad.

A continuación vamos a aplicaremos un filtro de imagen para eliminar pequeños puntos activos de ruido. Luego utilizaremos el resultado como una máscara y la aplicaremos a la imagen principal.

In [ ]:
fgmask = fgbg.apply(frame)
fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)

bg = cv.bitwise_and(frame, frame, mask=fgmask)
final = cv.bitwise_or(bg, cv.resize(background, (1280, 720)))

El resultado puede verse en dos ventanas. En una se muestra la aplicación del algoritmo MOG2 y en otra el resultado. Cabe destacar que cuando el objeto queda parado se comienza a convertir en fondo por lo que es eliminado por el filtro.

<div> <img src="../Images/2_ACTIVIDAD/f2.png" width="300"/> </div> 

<div> <img src="../Images/2_ACTIVIDAD/f1.png" width="300"/> </div> 

## Resultado final

En estos dos videos se muestra el resultado en tiempo real de mover un objeto. El fondo es eliminado y en su lugar podría ponerse una imagen.

In [4]:
from IPython.display import HTML

HTML("""
    <video alt="test" idth="320" height="240"  controls>
        <source src="../Images/2_ACTIVIDAD/ACTIVIDAD2.mp4" type="video/mp4">
    </video>
""")

In [5]:
from IPython.display import HTML

HTML("""
    <video alt="test" idth="320" height="240"  controls>
        <source src="../Images/2_ACTIVIDAD/ACTIVIDAD3.mp4" type="video/mp4">
    </video>
""")